# 스타일 변환 (Neural style transfer)

#### PIL(Python Image Library) 설치

In [ ]:
!pip install pillow

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

### 텐서플로와 다른 라이브러리 임포트


In [ ]:
import tensorflow as tf
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl

#matplotlib default property 설정
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time

#### GPU 메모리 설정 (프로그램에서 필요한만큼만 할당하는 방식)

In [ ]:
# 런타임에서 할당하는데 필요한 양만큼의 GPU 메모리를 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

### 스타일 변환할 이미지 가져오기

#### 이미지를 Tensor 타입에서 Pillow Image 객체로 변환

In [ ]:
def tensor_to_image(tensor):
    tensor = tensor*255 # [0,1]에서 [0,255]로 변환
    tensor = np.array(tensor, dtype=np.uint8) # numpy array로 변환
    
    # 배치에서 이미지가 1개인지 확인
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
        
    return PIL.Image.fromarray(tensor) # PIL 이미지로 변환

스타일 이미지와 컨텐츠 이미지 다운로드

In [ ]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')

# https://commons.wikimedia.org/wiki/File:Vassily_Kandinsky,_1913_-_Composition_7.jpg
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

#### 이미지 확인

이미지를 파일에서 읽어서 크기가 최대 512를 넘지 않게 resize

In [ ]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img) # encoded image bytes로 읽기
    img = tf.image.decode_image(img, channels=3) # 이미지 종류를 자동으로 탐지해서 Tensor로 반환
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32) # 이미지 모양 (가로, 세로)
    long_dim = max(shape)
    scale = max_dim / long_dim # 이미지 크기를 512로 resize하기 위한 비율

    new_shape = tf.cast(shape * scale, tf.int32) #새로운 이미지 크기
    img = tf.image.resize(img, new_shape) # 이미지 resize
    
    # 미리 배치 차원을 추가해서 신경망 입력 형태로 만들어 넣음
    img = img[tf.newaxis, :]
    return img

이미지 보여주기

In [ ]:
def imshow(image, title=None):
    # 배치인 경우 첫번째 차원을 없앰
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## TF-Hub의 스타일 변환 이용해보기

In [ ]:
import tensorflow_hub as hub
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
# 이미지를 Tensor 타입에서 Pillow Image 객체로 변환
tensor_to_image(stylized_image)

## 컨텐츠와 스타일 표현 정의

VGG19 모델 테스트 - 출력 모양 확인

In [ ]:
# Preprocesses a tensor or Numpy array encoding a batch of images
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape

Top 5 예측 가져오기

https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/decode_predictions

In [ ]:
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

VGG19 모델 테스트 - Top layer 제외하고 로드한 후 계층 이름 출력

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

계층 선택 - 컨텐츠 출력 계층과 스타일 출력 계층의 정의

In [ ]:
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

### 특징 추출 모델 정의

VGGNet19의 지정된 계층을 출력 계층으로 하는 모델을 정의

In [ ]:
def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

모델 생성 후 스타일 계층과 출력 확인

In [ ]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

#Look at the statistics of each layer's output
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

## 스타일 계산

스타일은 feature map의 상관관계로 표현할 수 있으며, feature map의 상관관계를 나타내는 행렬 중의 하나가 Gram Matrix이다.

Gram Matrix는 feature map의 outer product 형태로 다음과 같이 정의되며 간단히 `tf.linalg.einsum`로 계산할 수 있다.

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$



In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor) # Feature Map의 outer product
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations) # 평균을 구함

## 스타일과 컨텐츠 출력 모델 생성

스타일과 컨텐츠 텐서를 return하는 모델 생성

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    # 컨텐츠와 스타일 출력
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    # 스타일의 Gram Matrix 계산
    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    # 컨텐츠 딕셔너리
    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    # 스타일 딕셔너리
    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

모델을 생성하고 컨텐츠 이미지에 대한 계층 별 출력 확인
(스타일 계층의 Gram Matrix와 컨텐츠 계층의 컨텐츠 출력)

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())


## Gradient Descent 실행
스타일과 컨텐츠 타겟 생성

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

업데이트 할 이미지를 `tf.Variable` 타입으로 생성 (Noise로 초기화해도 되는데 여기서는 Content Image로 초기화 함)

In [ ]:
image = tf.Variable(content_image)

픽셀을 [0,1]로 Clipping

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

옵티마지저 생성 (논문에는 LBFGS를 권장하지만 `Adam` 도 잘 작동됨)

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

Loss 계산 (컨텐츠 Loss와 스타일 Loss를 가중 합산해서 Total Loss 정의)

In [ ]:
style_weight=1e-2
content_weight=1e4

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    
    # style loss 계산
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()]) # 계층 별 MSE의 합
    style_loss *= style_weight / num_style_layers # 가중치

    # content loss 계산
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()]) # 계층 별 MSE의 합
    content_loss *= content_weight / num_content_layers # 가중치
    loss = style_loss + content_loss
    return loss

이미지를 업데이트하기 위해  `tf.GradientTape` 사용

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

테스트로 3 스텝 실행

In [ ]:
train_step(image)
train_step(image)
train_step(image)
tensor_to_image(image)

이미지 생성을 위해 10 Epoch 동안 실행

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))
  
end = time.time()
print("Total time: {:.1f}".format(end-start))

## Total variation loss

고주파 잡음이 많이 있어서 부드럽게 만들어 주는 정규화 항 추가

In [ ]:
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

#### 컨텐츠 이미지의 Total Variation 계산 후 이미지로 출력

In [ ]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14,10))
plt.subplot(2,2,1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Original")

plt.subplot(2,2,2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2,2,3)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Styled")

plt.subplot(2,2,4)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Styled")

고주파 잡음이 얼마나 많이 생겼는지 확인
고주파는 기본적으로 edge detector이므로 Sobel edge detector의 출력과 유사한지 확인

In [ ]:
plt.figure(figsize=(14,10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1,2,1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel-edges")
plt.subplot(1,2,2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel-edges")

정규화를 위한 TV Loss 계산

In [ ]:
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

In [ ]:
total_variation_loss(image).numpy()

TensorFlow API도 있음

In [ ]:
tf.image.total_variation(image).numpy()

## Total Variation 정규화 항을 추가해서 이미지 생성

In [ ]:
total_variation_weight=30

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

이미지 초기화

In [ ]:
image = tf.Variable(content_image)

최적화 수행

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

최종 결과 저장

In [ ]:
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)